In [154]:
import pandas as pd 
import sklearn
import numpy as np
import nltk
import re

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

from sklearn import tree
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_selection import chi2

from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline

from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest

from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support


import gensim, logging
from gensim.models import Word2Vec
from scipy import sparse

from sklearn import svm, grid_search
import numpy as np
from sklearn.grid_search import GridSearchCV

In [155]:
from nltk.corpus import stopwords

In [156]:
stoplist = stopwords.words('english')

In [157]:
more_stopwords = "based private ltd co serive services amp /n /r ltd provide year company special  giving established various became 1987 range like every center best quality shop india indian complete range leading concern like time latest every one well-known also south delhi mumbai india indian bangalore hyderabad chennai"
stoplist += more_stopwords.split()

In [158]:
stoplist[0]

u'i'

In [159]:
import pandas as pd 
import sklearn
import numpy as np
import nltk
import re

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

from sklearn import tree
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_selection import chi2

from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline

from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest

from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support


import gensim, logging
from gensim.models import Word2Vec
from scipy import sparse

def loadData(filePath="dataset.csv"):
    data=pd.read_csv("/home/administrator/data/categories-data/Train-Data/fps-with-cat-train.csv") 
    data['CategoryFB'] = data['CategoryFB'].fillna(data['CategoryV2'])
    #data = data.dropna(subset=['Description'])
    #data = data.reset_index(drop=True)
    return data["Tag"],data["Name"],data["CategoryV2"]

def preProcessing(features):
    num_descs = features.size
    clean_wordlist = []
    clean_descs = []
    stops = set(stopwords.words('english'))
    more_stopwords = " since private ltd co based nwe n r \n \r amp st ncar nagar nto nand quot k provide service services year giving established various became 1987 range like every center best quality india indian complete range leading concern like time latest every one well-known also south delhi mumbai india indian bangalore hyderabad chennai"
    stops.update(more_stopwords.split())
    for i in range( 0, num_descs):
        letters_only = re.sub("[^a-zA-Z]", " ", features[i])
        #print letters_only
        words = letters_only.lower().split()
        words = [w.lower() for w in words if not w in stops]  
        clean_wordlist.append(words)
        clean_descs.append(" ".join(words))
    return clean_descs, clean_wordlist


def getDTMByTFIDF(features,nfeatures):
    tfIdf_vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1,2), min_df = 0, stop_words = 'english', max_features=nfeatures)
    dtm = tfIdf_vectorizer.fit_transform(features)
    return dtm,tfIdf_vectorizer

def featuresByChiSq(features,labels,nFeature=3000):
    chi2_model = SelectKBest(chi2,k=nFeature)
    dtm = chi2_model.fit_transform(features,labels)
    return dtm,chi2_model

In [160]:
from sklearn import svm, grid_search
import numpy as np
from sklearn.grid_search import GridSearchCV
def svc_param_selection(X, y, nfolds):
    Cs = [0.001, 0.01, 0.1, 1, 10]
    gammas = [0.001, 0.01, 0.1, 1]
    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(svm.SVC(kernel='linear', probability=True, class_weight='auto'), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search.best_params_

def crossValidate(document_term_matrix,labels,classifier="SVM",nfold=10):
    clf = None
    precision = []
    recall = []
    fscore = []
    
    if classifier == "RF":
        clf = RandomForestClassifier(class_weight='auto')
    elif classifier == "NB":
        clf = MultinomialNB()
    elif classifier == "SVM":
        clf = LinearSVC(class_weight='auto')
        #best_params = svc_param_selection(document_term_matrix, labels, nfold)
        #print best_params
        
    
    skf = StratifiedKFold(labels, n_folds=nfold)

    for train_index, test_index in skf:
        X_train, X_test = document_term_matrix[train_index], document_term_matrix[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        model = clf.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        p,r,f,s = precision_recall_fscore_support(y_test, y_pred, average='weighted')
        precision.append(p)
        recall.append(r)
        fscore.append(f)
        
    return round(np.mean(precision),3),round(np.mean(recall),3),round(np.mean(fscore),3)

In [161]:
tags, descs, labels = loadData()
print descs[0]
processed_descs, processed_descs_wordlist = preProcessing(descs)

Hanuman Enterprises India Pvt. Ltd.


In [162]:
len(labels)

34492

In [163]:
dtm,vect = getDTMByTFIDF(processed_descs,3000)

In [164]:
chisqDtm, chisqModel = featuresByChiSq(dtm,labels,2000)

In [165]:
precision, recall, fscore = crossValidate(chisqDtm,labels,"SVM",10)

In [141]:
print precision, recall, precision_recall_fscore_support

0.941 0.867 <function precision_recall_fscore_support at 0x7fe7bce175f0>


In [142]:
from sklearn.cross_validation import train_test_split

train_descs, test_descs = train_test_split(descs, test_size=0.1, 
                                           random_state=42)

train_labels, test_labels = train_test_split(labels, test_size=0.1, 
                                           random_state=42)

train_descs = train_descs.reset_index(drop=True)
test_descs = test_descs.reset_index(drop=True)
train_labels = train_labels.reset_index(drop=True)
test_labels = test_labels.reset_index(drop=True)

processed__train_descs, processed_train_descs_wordlist = preProcessing(train_descs)
processed_test_descs, processed_test_descs_wordlist = preProcessing(test_descs)

dtm_train,vect_train = getDTMByTFIDF(processed__train_descs,2000)
chisqDtmTrain, chisqModelTrain = featuresByChiSq(dtm_train,train_labels,2000)

clf = RandomForestClassifier(class_weight='auto')

In [143]:
from sklearn.cross_validation import train_test_split

train_descs, test_descs = train_test_split(descs, test_size=0.1, 
                                           random_state=42)

train_labels, test_labels = train_test_split(labels, test_size=0.1, 
                                           random_state=42)

train_descs = train_descs.reset_index(drop=True)
test_descs = test_descs.reset_index(drop=True)
train_labels = train_labels.reset_index(drop=True)
test_labels = test_labels.reset_index(drop=True)

processed__train_descs, processed_train_descs_wordlist = preProcessing(train_descs)
processed_test_descs, processed_test_descs_wordlist = preProcessing(test_descs)

dtm_train,vect_train = getDTMByTFIDF(processed__train_descs,1500)
chisqDtmTrain, chisqModelTrain = featuresByChiSq(dtm_train,train_labels,1000)

clf = LinearSVC(class_weight='auto')

In [144]:
model = clf.fit(chisqDtmTrain, train_labels)
dtm_test,vect_test = getDTMByTFIDF(processed_test_descs,1500)
chisqDtmTest, chisqModelTest = featuresByChiSq(dtm_test,test_labels,1000)
y_pred = model.predict(chisqDtmTest)
#predictions = model.predict_proba(chisqDtmTest)


p,r,f,s = precision_recall_fscore_support(test_labels, y_pred, average='weighted')

In [151]:
print p,r,f,s

print y_pred[245]

print test_labels[245]

print test_descs[245]

#labels.value_counts()

0.239505684564 0.132753623188 0.165201961609 None
BEAUTY & SPA
TOURISM
Kouser Travels


In [152]:
from collections import defaultdict
cdict = defaultdict(list)
for i, val in enumerate(labels):
    if val in cdict:
        cdict[val].extend(processed_descs_wordlist[i])
    else:
        value = list()
        cdict[val] = processed_descs_wordlist[i]

In [153]:
from collections import defaultdict
fq= defaultdict( int )
for w in cdict['BEAUTY & SPA']:
    fq[w] += 1
    
copy = []
for k,v in fq.items():
    copy.append((v, k))


copy = sorted(copy, reverse=True)

index = 0
for k in copy:
    if index == 50:
        break
    else:
        print '%s: %d' %(k[1], k[0])
    index = index+1

beauty: 930
salon: 709
spa: 285
parlour: 232
naturals: 158
unisex: 144
hair: 94
care: 94
family: 56
clinic: 51
saloon: 47
ladies: 45
products: 36
studio: 33
academy: 32
herbal: 31
centre: 29
road: 28
elegance: 27
pvt: 26
fantasy: 26
nikhar: 23
training: 22
health: 22
lounge: 21
avon: 21
parlor: 20
make: 17
zone: 16
spas: 16
lakme: 16
world: 15
new: 15
jailaxmi: 15
saloni: 14
fashion: 14
women: 12
wellness: 11
makeup: 11
day: 11
thai: 10
style: 10
sri: 10
skin: 10
professional: 10
ombre: 10
collection: 10
aura: 10
anokhi: 10
senhora: 9


In [113]:
from sklearn.metrics import classification_report
print classification_report(test_labels, y_pred)

                         precision    recall  f1-score   support

           ARCHITECTURE       0.00      0.00      0.00         7
                   ARTS       0.00      0.00      0.00         4
             AUTOMOTIVE       0.00      0.00      0.00        14
           BEAUTY & SPA       0.30      0.19      0.23       175
                  BLOGS       0.01      0.15      0.02        13
               CATERING       0.00      0.00      0.00         2
              CHEMICALS       0.00      0.00      0.00         4
           CONSTRUCTION       0.10      0.33      0.15         3
              EDUCATION       0.11      0.11      0.11       417
            ELECTRONICS       0.64      0.57      0.60       441
          ENTERTAINMENT       0.00      0.00      0.00         8
                  EVENT       0.00      0.00      0.00        20
                FASHION       0.35      0.13      0.19       311
                FLORIST       0.00      0.00      0.00         2
       FOOD & BEVERAGES 